<a href="https://colab.research.google.com/github/Awnder/rag-workshop/blob/main/rag-workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries

In [ ]:
#https://streamlit.io/
# used andrew's pinecone and groq api keys, with github access key from sbc

In [3]:
%pip install langchain langchain-community langchain-pinecone
%pip install openai groq pinecone-client unstructured[all-docs] unstructured_inference sentence-transformers docx2txt openpyxl onnx==1.16.1
%pip install pdfminer==20191125 pdfminer.six==20221105 pillow_heif pi-heif

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import PyPDFLoader, UnstructuredExcelLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredImageLoader, GithubFileLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import openai
import os
from dotenv import load_dotenv
from groq import Groq

c:\Users\andrew.shiroma\My Drive\R&I AI Chatbot\venv_r&i_assistant\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Initialize the HuggingFace Embeddings client

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\andrew.shiroma\AppData\Local\Temp\ipykernel_23960\619240441.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


# Load in the Data

In [17]:
def build_directory_document(file_path: str, loader: object):
    ''' create a Document object with metadata and page content '''
    return Document(
        metadata={
            "source": file_path
        },
        page_content=f"Source: {file_path}\n{loader.load()[0].page_content}"
    )

def build_github_doc(repo_name: str, gitdoc: list):
    ''' create a Document object with metadata and page content. github loader loads all documents so requires a different function'''
    return Document(
        metadata={
            "source": f"{repo_name}{gitdoc.metadata['path']}"
        },
        page_content=f"Source: {repo_name}{gitdoc.metadata['path']}\n{[gitdoc][0].page_content}" # convert the doc to a 1-element list to access the page_content
    )

In [38]:
load_dotenv()

# need to install tesseract in order to use the OCR (optical character recognition) image features
# https://github.com/langchain-ai/langchain/issues/9644

def process_directory(directory_path):
    ''' resursively processes all files in a directory '''
    data = []
    # other docs
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
                data.append(build_directory_document(file_path, loader))
            # elif file[-3:] == 'png':
            #     loader = UnstructuredImageLoader(file_path)
            #     data.append({"File": file_path, "Data": loader.load()})
            elif file.endswith(".docx"):
                loader = Docx2txtLoader(file_path)
                data.append(build_directory_document(file_path, loader))
            elif file.endswith(".xlsx"):
                loader = UnstructuredExcelLoader(file_path)
                data.append(build_directory_document(file_path, loader))

    return data

def process_github():
    ''' recursively processes all files in a github repo '''
    data = []
    repo_name = "reportingandinsights/common-code"

    loader = GithubFileLoader(
        repo=repo_name,
        branch="main",
        access_token=os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
        file_filter=lambda file_path: file_path.endswith(".md") or file_path.endswith(".txt") or file_path.endswith(".xml"),
    )
    
    for gitdoc in loader.load():
        data.append(build_github_doc(repo_name + "/", gitdoc))

    return data

directory_path = "./r&i_assistant_docs"
documents = []
# documents += process_directory(directory_path)
documents += process_github()


In [22]:
data = []
loader = UnstructuredExcelLoader('./r&i_assistant_docs/Dictionaries Dashboard Source Files.xlsx')
data.append(build_directory_document('./r&i_assistant_docs/Dictionaries Dashboard Source Files.xlsx', loader))
print(data)
# vectorstore.add_documents(data)

[Document(metadata={'source': './r&i_assistant_docs/Dictionaries Dashboard Source Files.xlsx'}, page_content='Source: ./r&i_assistant_docs/Dictionaries Dashboard Source Files.xlsx\nSource/Table Type Workspace Name Dataflow Name (Source Name) Table Name (Entity Name) Owner (Bus/Dev) Last Model / Catalog Update Table/Entity Description Table/Entity Grain Dependencies Custom Tables (DATA) HC Data Warehouse Metrics Metrics VenueAttendanceMultiplier Ron Brown 2024-09-27 00:00:00 Table is used to store Multiplier rows by VenueID, CampusID, MultiplierCategory and Multiiplier. A multiplier can be updated by adding a Multiplier or changing A Multiplier. This is done in the back-end people PersonEngagement\', \'custom DimPerson\', \'custom Person_Spouse\' Custom Tables (DATA) HC Data Warehouse Metrics Metrics vwFactMetricsOnlineAttendance Ron Brown 2024-09-27 00:00:00 Mirrors Metrics.FactTrendMetrics table with added VenueID, MultiplierCategory & Multiplier. Other columns not related to attendan

In [39]:
documents

[Document(metadata={'source': 'reportingandinsights/common-code/Conditionally Hide Unwanted Visuals.md'}, page_content="Source: reportingandinsights/common-code/Conditionally Hide Unwanted Visuals.md\n# Conditionally Hide Unwanted Visuals in Power Bi\n\n## use a treemap visual to show white when certain slicers are selected. example below\n```\nM_SlicerCheck = \nCALCULATE(\n\t\tIF(\n\t\t//condition to activate\n\t\t\tISFILTERED('Dim RC_section (Spiritual)'[Spiritual Slicer]) \n\t\t,0 //display zero if a slicer is selected. this will make the treemap invisible. the final results table is filtered to SlicerCheck = 0\n        ,1\n        )\n\t, ALLSELECTED ( 'custom_DimSHAPE') \n)\n```\n\n## treemap instructions\n1) create measure (like above) for a slicer (M_SlicerCheck in example)\n2) make a treemap visual and make the slicer white (background color)\n3) use conditional formatting to tie the color display to the measure value\n4) Layer the treemap visual on top of the unwanted visuals\n

# Inserting Data Into Pinecone

In [6]:
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "ri-assist"

namespace = "" # if using the "vectorstore.add_documents" method, it upserts docs w/o namespace

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [20]:
# for idx, document in enumerate(documents):
#     print("Processing document: ", idx)
#     vectorstore_from_documents = PineconeVectorStore.from_documents(
#         [document],
#         embeddings,
#         index_name=index_name,
#         namespace=namespace
#     )

# add all documents at once. this seems to be faster than the above
vectorstore.add_documents(documents=documents) 

['48d65a82-e61c-41b9-965b-7c0cef923400',
 '7dc58f2e-b5dd-45b0-8e3d-25a3bde5de38',
 '5410d4df-8151-44de-a2c6-017528b0d3b7',
 '2fcfe5f0-e78c-4517-a459-68dec2c2ee2f',
 '4ed4cd97-2cda-480e-ad88-b7aaf21cd32e',
 '5b931356-12f3-4979-8b5a-418b51969360',
 'c1fbb0b6-bc4a-4cd2-aac4-c001f7fe1b67',
 '5cdd9349-f318-4b45-9ecb-988754bf548b',
 'ca93da95-7a07-4d5a-9e54-cf0ee919d003',
 'd7f96e65-b33d-438b-bdea-2827051ac123',
 'dcc4b01e-f53e-44c9-80cd-5de22196bcca',
 '5cd2558b-02bb-43d2-a533-fec0671c3f40',
 'e68a14de-e80d-4d1b-96d1-fa35a367c249',
 'e81f2a94-86e5-46d3-af1d-35878344b8c2',
 '43da74db-4538-4fed-8fd3-937c43ec02d1',
 '7bcc8b50-aaaa-4b07-9c84-6b238bf45c25',
 '58398934-0e18-42b4-8150-b53cc7c0b240',
 '8a78cd6a-04cf-4651-b6f8-57b5ed4e8b34',
 'b2689aba-1f07-4d08-803b-cd33d4c3716d',
 '75106ccf-af54-4f25-a5ff-b568a87518f4',
 '7f7500f3-a427-496b-b825-e89360dc480d',
 '191c1c8e-1333-4a42-ac4b-9a563b8eb653',
 'a90fff99-57bd-48ae-a6d2-ebc468bd05c8',
 '425ecdbd-41f3-4d91-bf97-c43eabafb298',
 '5faf9f24-d817-

In [ ]:
# doesnt seem to work as intended. I think dramatically writes to pinecone way more than necessary
# import multiprocessing

# def add_documents_batch(batch):
#     """Adds a batch of documents to the Pinecone index."""
#     vectorstore.add_documents(batch)

# def create_multiprocess_pool(documents, num_processes=None):
#     if num_processes is None:
#         num_processes = multiprocessing.cpu_count() # use all available cores
    
#     batch_size = len(documents) // num_processes
#     batches = [documents[i:i+batch_size] for i in range(0, len(documents), batch_size)]

#     with multiprocessing.Pool(processes=num_processes) as pool:
#         pool.map(add_documents_batch, batches) # requires .map(function, iterable)
#         print('pooling: ' + batches)

# create_multiprocess_pool(documents)

# BATCH_SIZE = 1
# for i in range(0, len(documents), BATCH_SIZE):
#     batch = documents[i:i+BATCH_SIZE]
#     vectorstore.add_documents(batch)
#     print(f"Inserting batch {i // BATCH_SIZE + 1} of {len(documents) // BATCH_SIZE + 1}")

# Initialize the Groq client

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)

In [7]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

groq_client = Groq(api_key=groq_api_key)

# Perform RAG

In [8]:
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [9]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [41]:
# construct query, embed it, get pinecone vector match, and grab contexts
query = "what do i do if i want to update the data dictionary?"

query_embed = get_huggingface_embeddings(query)
top_matches = pinecone_index.query(vector=query_embed.tolist(), top_k=5, include_metadata=True, namespace=namespace)
contexts = [item['metadata']['text'] for item in top_matches['matches']]
augmented_query = "<CONTEXT>\n" + "\n-------\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\nMY QUESTION:\n" + query


In [42]:
system_prompt = f'''
You are a confident expert at understanding and explaining business technology and development to interns and volunteers. Let's take this step-by-step:
First, answer any questions based on the data provided and always consider the context of the question, providing the most accurate and relevant information when forming a response.
Second, if unsure of anything, mention it in the response and provide a suggestion for further research. 
Third, suggestions can include web search, other document context, or asking questions to supervisors Kirk Yamada and Jeremiah Chuang.
Finally, cite your sources below your response only.'''

llm_response = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",  "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content
print(response)

To update the data dictionary, you should follow these steps:

1. **Identify the relevant data dictionary file**: Based on the provided context, there are multiple data dictionaries, such as R&IKPIDictionary.xlsx, KPIDictionary, FieldDictionary, and others.
2. **Locate the correct sheet or file**: Find the specific sheet or file that contains the data you want to update, such as the "Explanatory Notes for Reports" sheet in R&IKPIDictionary.xlsx or the "DataDictionary" file.
3. **Understand the data structure and conventions**: Familiarize yourself with the structure, formatting, and any specific conventions used in the data dictionary.
4. **Make the necessary updates**: Carefully review and update the relevant rows, columns, or fields with the new information. Ensure that any changes are consistent with the existing data and formatting conventions.
5. **Document changes and provide explanations**: If you're updating metric-level information or explanatory notes, ensure that you documen